In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os

In [3]:
import tensorflow as tf
print(tf.__version__)

2.3.2


In [4]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

## Data preprocessing

### Data download

In [6]:
URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

In [7]:
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=URL, extract=True)

68608000/68606236 [==============================] - 1s 0us/step


In [8]:
path_to_zip

'/home/ec2-user/.keras/datasets/cats_and_dogs.zip'

In [11]:
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

In [12]:
PATH

'/home/ec2-user/.keras/datasets/cats_and_dogs_filtered'

In [13]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [14]:
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

In [15]:
train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

Found 2000 files belonging to 2 classes.


In [16]:
validation_dataset = image_dataset_from_directory(
    validation_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

Found 1000 files belonging to 2 classes.


In [17]:
class_names = train_dataset.class_names

In [18]:
class_names

['cats', 'dogs']

As the original dataset doesn't contain a test set, you will create one. To do so, determine how many batches of data are available in the validation set using ```tf.data.experimental.cardinality```, then move 20% of them to a test set.

In [20]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [22]:
print(f'Number of validation batches: {tf.data.experimental.cardinality(validation_dataset)}')
print(f'Number of test batches: {tf.data.experimental.cardinality(test_dataset)}')

Number of validation batches: 26
Number of test batches: 6


### Configure the dataset for performance

In [24]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [25]:
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Use data augmentation

In [26]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

### Rescale pixel values

In [28]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [29]:
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

## Create the base model from the pre-trained convnets

In [30]:
IMG_SHAPE = IMG_SIZE + (3,)

In [31]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

9412608/9406464 [==============================] - 0s 0us/step


In [40]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 5, 5, 1280)


## Feature extraction

### Freeze the convolutional base

In [33]:
base_model.trainable = False

### Important note about BatchNormalization layers

### Add a classification head

In [35]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)

In [41]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)

In [42]:
inputs = tf.keras.Input(shape=(160, 160, 3))

x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)

outputs = prediction_layer(x)

model = tf.keras.Model(inputs, outputs)

### Compile the model

In [43]:
base_learning_rate = 0.0001

In [44]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [48]:
len(model.trainable_variables)

2

### Train the model

In [52]:
initial_epochs = 3

In [50]:
loss0, accuracy0 = model.evaluate(validation_dataset)

26/26 [==============================] - 10s 369ms/step - loss: 0.6955 - accuracy: 0.5718


In [51]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 0.70
initial accuracy: 0.57


In [53]:
history = model.fit(
    train_dataset,
    epochs=initial_epochs,
    validation_data=validation_dataset
)

Epoch 1/3
63/63 [==============================] - 39s 616ms/step - loss: 0.6885 - accuracy: 0.6000 - val_loss: 0.4928 - val_accuracy: 0.7290
Epoch 2/3
63/63 [==============================] - 38s 607ms/step - loss: 0.5416 - accuracy: 0.7100 - val_loss: 0.3520 - val_accuracy: 0.8403
Epoch 3/3
63/63 [==============================] - 38s 596ms/step - loss: 0.4353 - accuracy: 0.7815 - val_loss: 0.2826 - val_accuracy: 0.8886


## Fine tuning

### Un-freeze the top layers of the model


In [56]:
base_model.trainable = True

In [57]:
print(f"Number of layers in the base model: {len(base_model.layers)}")

Number of layers in the base model: 155


In [58]:
fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

### Compile the model

In [59]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
    metrics=['accuracy']
)

### Continue training the model

In [62]:
fine_tune_epochs = 3

In [63]:
total_epochs =  initial_epochs + fine_tune_epochs

In [65]:
history_fine = model.fit(
    train_dataset,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=validation_dataset
)

Epoch 3/6
63/63 [==============================] - 59s 939ms/step - loss: 0.2042 - accuracy: 0.9065 - val_loss: 0.0690 - val_accuracy: 0.9839
Epoch 4/6
63/63 [==============================] - 55s 881ms/step - loss: 0.1286 - accuracy: 0.9460 - val_loss: 0.0516 - val_accuracy: 0.9889
Epoch 5/6
63/63 [==============================] - 55s 876ms/step - loss: 0.1114 - accuracy: 0.9565 - val_loss: 0.0486 - val_accuracy: 0.9790
Epoch 6/6
63/63 [==============================] - 55s 880ms/step - loss: 0.1169 - accuracy: 0.9510 - val_loss: 0.0447 - val_accuracy: 0.9814


In [66]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

### Evaluation and prediction

In [68]:
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy : {accuracy}')

6/6 [==============================] - 2s 334ms/step - loss: 0.0423 - accuracy: 0.9844
Test accuracy : 0.984375


In [69]:
image_batch, label_batch = test_dataset.as_numpy_iterator().next()

In [70]:
predictions = model.predict_on_batch(image_batch).flatten()

In [71]:
predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

In [73]:
print(f'Predictions:\n {predictions.numpy()}')
print(f'Labels:\n {label_batch}')

Predictions:
 [0 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 1 0 0 1]
Labels:
 [0 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 1 0 0 1]
